In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def mu_drift(x,t):
    return -x       # the drift

def Dif(x,t):
    return 1        # diffusion coefficient

def get_arraytot(psi):
    
    Nx            = np.size(psi)
    psi_tot       = np.zeros(Nx+2).astype('float64')
    
    psi_tot[1:-1] = psi

    psi_tot[0]    = psi_tot[1] -  ( psi_tot[2] - psi_tot[1] )
    psi_tot[-1]   = psi_tot[-2] +  ( psi_tot[-2] - psi_tot[-3] )
    
    return psi_tot  # add ghost cells to domain boundary, with linearly interpolated values

In [3]:
# Parameters of the simulation

xbeg = -10  # xmin
xend = 10   # xmax
Nx   = 100  # number of cells in active zone
dt_0 = 1e-3 # initial time step
CFL  = 0.9  # CFL stability factor
tend = 10   # end time of simulation
n_it = 10   # save snapshot at each n_it'h number of step

In [4]:
# define the grid, grid spacing and initial distribution of the variable psi

xp       = np.linspace(xbeg,xend,Nx)
dx       = xp[1] - xp[0]
psi_init = np.exp(-(xp-3)**4)  

#***************************************************************************

# define auxiliary arrays for computation***********************************

dmxp_dx   = np.zeros_like(psi_init)
d2Dxp_dx2 = np.zeros_like(psi_init)
xp_tot    = get_arraytot(xp)  # addd ghost cells

#***************************************************************************


#start computation**********************************************************

t             = 0
psi           = psi_init
dt            = dt_0
it            = 0
plt.figure(figsize=(6,4))

while (t<tend):  # beginning of time loop
    
    if it % n_it == 0:
        plt.xlabel(r"$x$")
        plt.ylabel(r"$\psi$")
        plt.xlim([xbeg,xend])
        plt.ylim([-4,4])
        plt.plot(xp,psi)
        plt.savefig("ps"+str(it)+".png",dpi=300)
        plt.clf()
        plt.close()
    
    psi_tot     = get_arraytot(psi)  # add ghost cells
    
    
    mu_tot      = [mu_drift(x,t) for x in xp_tot]
    mu_x_psitot = mu_tot * psi_tot
    dmxp_dx     = (mu_x_psitot[2:Nx+2]-mu_x_psitot[0:Nx])/(2*dx)
    
    D_tot       = [Dif(x,t) for x in xp_tot]
    D_x_psitot  = D_tot * psi_tot
    d2Dxp_dx2   = (D_x_psitot[2:Nx+2]-2*D_x_psitot[1:Nx+1]+D_x_psitot[0:Nx])/(2*dx**2)
    
    dt          = CFL * dx**2 / (2 * (np.max(D_tot)) )  # deetrmine dt with CFL stability criteria
    
    
    
    psi         += dt * ( - dmxp_dx + d2Dxp_dx2 )       # update the distribution
    
    t           += dt
    it          = it + 1